In [20]:
# tx에 U로 시작하는 것들은 어떤 규칙으로 적혀있는지 확인
# U = [i for i in df['tx'] if str(i).startswith('U')]

#print(U)

#print(df['tx'].dtype)

In [3]:
# icd_10 분류에는 C42 가 들어있지 않다.
# 아래 코드로 확인해보니, 현재 데이터에는 C42가 존재하지 않는다.

# checklist = df['icd_10'].tolist()

# for i in checklist:
#     if str(i).startswith('C42'):
#         break
#     else:
#         continue

# 변수설명
**SERIAL_ID**
* 환자별 id

**sex** 
* 성별 : 남=1, 여=2

**icd_10**
* icd-10 국제 규격에 맞는 질병 분류

**seercode**
* 병기 : 전이가 얼마나 되었는지를 나타내는 코드
* 0~1 : 전이 X 
* 2,3,4,5,7 : 전이 O 
* 9 & Null : 9와 null 값들은 전이가 되었는지 아닌지 불분명한 값들이기 때문에 제거함.

**age**
* 나이

**event_inc**
* event 발생 O: 1
* event 발생 X or 모름: 0
* event 가 발생하지 않았거나 모르는 경우는 event_inc를 0으로 코딩하고, death 날짜는 202401 로 코딩

**stime**
* 생존기간 (단위/ 개월)

**tx_1,2,3,4,5**
* 수술, 화학요법, 방사선요법, 면역요법, 호르몬요법 순으로 더미변수화함.

**seer_TF**
* seercode가 전이의 정도(전이가 얼마나 되었는지)를 세세히 표기했다면, seer_TF는 전이의 유무만을 나타냄.

* seercode==0 | seercode==1 ---> 0 (전이 X)
* seercode== 1,2,3,4,5 ---> 1 (전이 O )


---

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('C:/Users/user/Desktop/Survival_Data/FINAL.csv')
main_path = "C:/Users/user/Desktop/Survival_Data/DISEASE/"

# 6대암 분류

## 악성신생물들만 포함
* 양성종양, 제자리암종, 형태불명의 종양은 미포함

**제자리암종(전암병변)**

* 어떤 장기의 정상조직에서 암으로 발전할 수 있는 중간 단계의 병변들이 있는데, 이것을 '전암병변'이라고 함. 즉, 암 전단계의 병변이라고 함.

* 우리 연구의 목적은 암발생 환자의 생존율을 예측하는 것이 목적이므로, 전암병변은 포함시키지 않는 것이 옳다고 생각.

## 정리
1. 악성종양에 대해서만 분석 -> mcode는 제외
2. 병기는 전이유무 (True/ False)로 처리
-> 전이가 null 인 경우는 임의로 False로 처리.
3. 프레일티, 코풀라 모형을 실제 데이터에 비교한다는 개념으로 가보자

In [5]:
# 위암
# 위에 발생한 악성신생물
stomach_list = ["C160", "C161", "C162", 'C163', "C164", "C165", "C166", "C169", "C16"]

stomach_df = df[(df['icd_10']=="C160")|
               (df['icd_10']=="C161")|
               (df['icd_10']=="C162")|
               (df['icd_10']=="C163")|
               (df['icd_10']=="C164")|
               (df['icd_10']=="C165")|
               (df['icd_10']=="C166")|
               (df['icd_10']=="C169")|
               (df['icd_10']=="C16")]

stomach_df.head()
label_encoder = LabelEncoder()
stomach_df.loc[:,'gu_encoded'] = label_encoder.fit_transform(stomach_df['gu'])

C:\Users\user\AppData\Local\Temp\ipykernel_22808\934517525.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stomach_df.loc[:,'gu_encoded'] = label_encoder.fit_transform(stomach_df['gu'])


In [7]:
stomach_df =stomach_df.reset_index(drop=True)
stomach_df

,SERIAL_ID,sex,age,icd_10,gu,seercode,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5,seer_TF,gu_encoded
0,142,1,59,C163,해운대구,3.0,1,0,0,0,0,0,0,1,15
1,142,1,59,C163,해운대구,3.0,1,0,0,0,0,0,0,1,15
2,328,2,72,C160,사하구,2.0,1,0,0,0,0,0,0,1,9
3,365,1,41,C161,사하구,3.0,1,160,1,0,1,0,0,1,9
4,478,1,59,C163,강서구,1.0,1,168,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41551,364285,1,37,C160,강서구,1.0,0,25,1,0,0,0,0,0,0
41552,364314,1,77,C162,동구,7.0,0,25,0,1,0,0,0,1,4
41553,364320,2,66,C163,기장군,1.0,0,25,1,0,0,0,0,0,2
41554,364341,2,75,C169,연제구,2.0,0,25,0,0,0,0,0,1,12


In [8]:
stomach_df.to_csv(main_path+'stomach.csv')

In [10]:
import json

# 원본 문자열과 숫자 값을 매핑한 dictionary 생성
gu_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
output_path = "C:/Users/user/Desktop/Survival_Data/GU_json/stomach_gu.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(gu_mapping, f, ensure_ascii=False, indent=4)

TypeError: Object of type int32 is not JSON serializable

In [5]:
stomach_df.head()

,SERIAL_ID,sex,age,icd_10,gu,seercode,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5,seer_TF
0,142,1,59,C163,해운대구,3.0,1,0,0,0,0,0,0,1
1,142,1,59,C163,해운대구,3.0,1,0,0,0,0,0,0,1
2,328,2,72,C160,사하구,2.0,1,0,0,0,0,0,0,1
3,365,1,41,C161,사하구,3.0,1,160,1,0,1,0,0,1
4,478,1,59,C163,강서구,1.0,1,168,1,0,0,0,0,0


In [6]:
print(len(stomach_df))

41556


In [7]:
# 간암
# 간에 발생한 악성 신생물들

liver_list = ["C220", "C221", "C222", "C223", "C224", "C227", "C229"]

liver_df = df[(df['icd_10']=="C220")|
               (df['icd_10']=="C221")|
               (df['icd_10']=="C222")|
               (df['icd_10']=="C223")|
               (df['icd_10']=="C224")|
               (df['icd_10']=="C227")|
               (df['icd_10']=="C229")|
              (df['icd_10']=="C22")]
label_encoder = LabelEncoder()
liver_df['gu_encoded'] = label_encoder.fit_transform(liver_df['gu'])
display(liver_df.head())
print(f'{len(liver_df)}')

,SERIAL_ID,sex,age,icd_10,gu,seercode,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5,seer_TF
40,833,2,65,C220,해운대구,1.0,1,46,0,0,0,0,0,0
58,1494,1,61,C220,금정구,2.0,1,0,0,0,0,0,0,1
96,2571,1,50,C220,동래구,1.0,1,175,1,1,0,0,0,0
99,2631,2,73,C229,부산진구,7.0,1,3,0,0,0,0,0,1
125,3532,1,50,C229,사하구,1.0,1,118,1,0,0,0,0,0


25324


In [8]:
liver_df =liver_df.reset_index(drop=True)
liver_df
liver_df.to_csv(main_path+'liver.csv')

In [9]:
liver_df

,SERIAL_ID,sex,age,icd_10,gu,seercode,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5,seer_TF
0,833,2,65,C220,해운대구,1.0,1,46,0,0,0,0,0,0
1,1494,1,61,C220,금정구,2.0,1,0,0,0,0,0,0,1
2,2571,1,50,C220,동래구,1.0,1,175,1,1,0,0,0,0
3,2631,2,73,C229,부산진구,7.0,1,3,0,0,0,0,0,1
4,3532,1,50,C229,사하구,1.0,1,118,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25319,364194,1,70,C220,부산진구,2.0,0,25,0,1,0,0,0,1
25320,364219,1,72,C220,금정구,1.0,0,25,1,0,0,0,0,0
25321,364236,1,55,C220,연제구,1.0,1,1,0,0,0,0,0,0
25322,364268,1,68,C220,연제구,1.0,0,25,1,0,0,0,0,0


In [11]:
# 대장암 (결장의 악성신생물 + 직장의 악성신생물)
colon_list = ["C18", "C180", "C181", "C182", "C183", "C184", "C185", "C186", "C187", "C188", "C189",  "C20"]

colon_df = df[(df['icd_10']=="C18")|
               (df['icd_10']=="C180")|
               (df['icd_10']=="C181")|
               (df['icd_10']=="C182")|
               (df['icd_10']=="C183")|
               (df['icd_10']=="C184")|
               (df['icd_10']=="C185")|
              (df['icd_10']=="C186")|
              (df['icd_10']=="C187")|
              (df['icd_10']=="C189")|
              (df['icd_10']=="C20")]

In [12]:
label_encoder = LabelEncoder()
colon_df['gu_encoded'] = label_encoder.fit_transform(colon_df['gu'])

colon_df =colon_df.reset_index(drop=True)
colon_df.to_csv(main_path+'colon.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_22808\654666098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colon_df['gu_encoded'] = label_encoder.fit_transform(colon_df['gu'])


In [13]:
colon_df

,SERIAL_ID,sex,age,icd_10,gu,seercode,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5,seer_TF,gu_encoded
0,996,2,66,C187,금정구,4.0,1,64,1,1,0,0,0,1,1
1,1441,2,46,C180,서구,1.0,0,298,1,1,0,0,0,0,10
2,1658,1,64,C184,영도구,1.0,1,241,1,1,0,0,0,0,13
3,2428,1,47,C20,사상구,2.0,1,104,1,1,0,0,0,1,8
4,3078,2,57,C20,해운대구,1.0,0,296,1,0,0,0,0,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33971,364369,1,82,C20,동래구,2.0,1,2,1,0,0,0,0,1,5
33972,364371,1,64,C180,해운대구,4.0,0,25,1,0,0,0,0,1,15
33973,364376,1,52,C185,남구,1.0,0,25,1,0,0,0,0,0,3
33974,364386,1,56,C182,해운대구,4.0,0,25,1,1,0,0,0,1,15


In [13]:
# 폐암(기관지 및 폐의 악성 신생물)

lung_list = ["C34", "C340", "C341", "C342", "C343", "C348", "C349"]

lung_df = df[(df['icd_10']=="C34")|
               (df['icd_10']=="C340")|
               (df['icd_10']=="C341")|
               (df['icd_10']=="C342")|
               (df['icd_10']=="C343")|
               (df['icd_10']=="C348")|
              (df['icd_10']=="C349")]

In [14]:
label_encoder = LabelEncoder()
lung_df['gu_encoded'] = label_encoder.fit_transform(lung_df['gu'])

lung_df =lung_df.reset_index(drop=True)
lung_df.to_csv(main_path+'lung.csv')

In [15]:
lung_df

,SERIAL_ID,sex,age,icd_10,gu,seercode,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5,seer_TF
0,559,1,68,C349,강서구,7.0,1,3,0,0,0,0,0,1
1,704,2,70,C341,부산진구,7.0,1,58,0,1,0,0,0,1
2,711,1,69,C341,사상구,7.0,1,12,0,0,0,0,0,1
3,805,2,79,C349,서구,2.0,1,0,0,0,0,0,0,1
4,1080,1,64,C341,금정구,7.0,1,133,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31485,364309,1,80,C349,동래구,7.0,1,5,0,0,0,0,0,1
31486,364326,1,69,C343,금정구,7.0,1,5,0,0,0,0,0,1
31487,364328,1,57,C341,금정구,7.0,0,25,0,1,0,0,0,1
31488,364348,2,72,C343,서구,7.0,1,3,0,0,1,0,0,1


In [16]:
# 유방암(유방에 발생한 악성신생물)
breast_list=["C500", "C501", "C502", "C503", "C504", "C505", "C506", "C508", "C509"]

breast_df = df[(df['icd_10']=="C500")|
               (df['icd_10']=="C501")|
               (df['icd_10']=="C502")|
               (df['icd_10']=="C503")|
               (df['icd_10']=="C504")|
               (df['icd_10']=="C505")|
              (df['icd_10']=="C506")|
              (df['icd_10']=="C508")|
              (df['icd_10']=="C509")|
              (df['icd_10']=="C50")]

In [17]:
label_encoder = LabelEncoder()
breast_df['gu_encoded'] = label_encoder.fit_transform(breast_df['gu'])

breast_df =breast_df.reset_index(drop=True)
breast_df.to_csv(main_path+'breast.csv')

In [18]:
breast_df

,SERIAL_ID,sex,age,icd_10,gu,seercode,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5,seer_TF
0,73,2,46,C504,동래구,2.0,1,100,1,1,0,0,0,1
1,493,2,33,C504,부산진구,1.0,1,125,1,1,0,0,0,0
2,644,2,45,C502,남구,1.0,0,300,1,1,0,0,0,0
3,1046,2,45,C508,남구,7.0,0,299,1,1,0,0,0,1
4,1214,2,44,C509,사하구,7.0,1,46,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25891,364361,2,42,C504,사상구,1.0,0,25,1,1,0,0,0,0
25892,364366,2,52,C504,부산진구,1.0,0,25,1,0,1,0,1,0
25893,364368,2,56,C508,동래구,7.0,0,25,0,1,0,0,1,1
25894,364373,2,56,C509,동래구,1.0,0,25,1,1,0,0,0,0


In [19]:
# 자궁경부암
cervical_list= ["C530", "C531", "C538", "C539"]

cervical_df = df[(df['icd_10']=="C530")|
                (df['icd_10']=="C531")|
                (df['icd_10']=="C538")|
                (df['icd_10']=="C539")]

In [20]:
label_encoder = LabelEncoder()
cervical_df['gu_encoded'] = label_encoder.fit_transform(cervical_df['gu'])

cervical_df =cervical_df.reset_index(drop=True)
cervical_df.to_csv(main_path+'cervical.csv')

In [21]:
cervical_df

,SERIAL_ID,sex,age,icd_10,gu,seercode,event_inc,stime,tx_1,tx_2,tx_3,tx_4,tx_5,seer_TF
0,82,2,31,C539,남구,4.0,1,62,1,1,0,0,0,1
1,82,2,31,C539,남구,4.0,1,62,1,1,0,0,0,1
2,378,2,75,C539,남구,1.0,1,31,0,1,0,0,0,0
3,966,2,58,C539,해운대구,2.0,1,57,0,1,0,0,0,1
4,2105,2,59,C539,해운대구,1.0,1,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5992,363627,2,33,C539,사하구,1.0,0,25,1,0,0,0,0,0
5993,363672,2,42,C539,동래구,4.0,0,25,1,1,1,0,0,1
5994,363694,2,46,C539,남구,4.0,0,25,0,1,1,0,0,1
5995,363734,2,44,C539,사하구,1.0,0,25,1,0,0,0,0,0


In [22]:
len(stomach_df)+len(cervical_df)+len(breast_df)+len(liver_df)+len(breast_df)+len(colon_df)+len(lung_df)

190135